In [1]:

from pyphm.datasets.ims import ImsDataLoad
from pathlib import Path
import pandas as pd
import os
import numpy as np
import time
import datetime

%load_ext autoreload
%autoreload 2

In [2]:
root_dir = Path.cwd().parent
print(root_dir)
path_data_raw_folder = Path(root_dir / 'data' )
print(path_data_raw_folder)

/home/tim/Documents/PyPHM
/home/tim/Documents/PyPHM/data


In [3]:
ims = ImsDataLoad(path_data_raw_folder, 'ims', download=False)

type(root) =  <class 'pathlib.PosixPath'>
/home/tim/Documents/PyPHM/data/ims


In [15]:
path_1st_folder = ims.path_1st_folder
path_2nd_folder = ims.path_2nd_folder
path_3rd_folder = ims.path_3rd_folder

In [19]:
file_list = sorted(os.listdir(path_3rd_folder))
file_list[0]

'2004.03.04.09.27.46'

In [20]:
file_name = file_list[0]
print(file_name)

col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv(path_3rd_folder / file_name, sep='\t', names=col_names)
df.head()

2004.03.04.09.27.46


,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8
0,0.034,0.264,0.039,-0.046,NaN,NaN,NaN,NaN
1,0.103,0.083,-0.061,-0.012,NaN,NaN,NaN,NaN
2,0.095,-0.039,-0.007,0.039,NaN,NaN,NaN,NaN
3,0.000,0.110,0.022,-0.002,NaN,NaN,NaN,NaN
4,0.005,0.154,-0.127,-0.020,NaN,NaN,NaN,NaN


In [6]:
# columns to add
# ['id', 'run', 'start_time_unix', 'date_time', 'unix_time', 'file_name']

df['id'] = "1_" + file_name
df['run'] = 1
df.head()

,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8,id,run
0,-0.022,-0.039,-0.183,-0.054,-0.105,-0.134,-0.129,-0.142,1_2003.10.22.12.06.24,1
1,-0.105,-0.017,-0.164,-0.183,-0.049,0.029,-0.115,-0.122,1_2003.10.22.12.06.24,1
2,-0.183,-0.098,-0.195,-0.125,-0.005,-0.007,-0.171,-0.071,1_2003.10.22.12.06.24,1
3,-0.178,-0.161,-0.159,-0.178,-0.100,-0.115,-0.112,-0.078,1_2003.10.22.12.06.24,1
4,-0.208,-0.129,-0.261,-0.098,-0.151,-0.205,-0.063,-0.066,1_2003.10.22.12.06.24,1


In [26]:
def process_raw_csv(path_run_folder, file_name, sample_index, run_no, col_names, sample_freq=20480.0):

    df = pd.read_csv(path_run_folder / file_name, sep='\t', names=col_names)

    df['id'] = "1_" + str(sample_index)
    df['run'] = run_no
    df['file'] = file_name
    df['time'] = np.linspace(0.0, len(df)/sample_freq, len(df))

    return df[['id', 'file', 'run', 'time',] + col_names]

In [27]:
df1 = process_raw_csv(path_3rd_folder, file_name, 0, 3, col_names, sample_freq=20480.0)
df1.head()

,id,file,run,time,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8
0,1_0,2004.03.04.09.27.46,3,0.000000,0.034,0.264,0.039,-0.046,NaN,NaN,NaN,NaN
1,1_0,2004.03.04.09.27.46,3,0.000049,0.103,0.083,-0.061,-0.012,NaN,NaN,NaN,NaN
2,1_0,2004.03.04.09.27.46,3,0.000098,0.095,-0.039,-0.007,0.039,NaN,NaN,NaN,NaN
3,1_0,2004.03.04.09.27.46,3,0.000146,0.000,0.110,0.022,-0.002,NaN,NaN,NaN,NaN
4,1_0,2004.03.04.09.27.46,3,0.000195,0.005,0.154,-0.127,-0.020,NaN,NaN,NaN,NaN


In [8]:
# convert start_time to unix timestamp
start_time_unix = time.mktime(
    datetime.datetime.strptime(file_name, "%Y.%m.%d.%H.%M.%S").timetuple()
)
start_time

1066849584.0

In [9]:
date_nice_format = datetime.datetime.fromtimestamp(start_time).strftime(
    "%Y-%m-%d %H:%M:%S"
)  # reformat date
date_nice_format


'2003-10-22 12:06:24'

In [7]:
date_list = sorted(os.listdir(path_1st_folder))

In [10]:
date_list[:10]

['2003.10.22.12.06.24',
 '2003.10.22.12.09.13',
 '2003.10.22.12.14.13',
 '2003.10.22.12.19.13',
 '2003.10.22.12.24.13',
 '2003.10.22.12.29.13',
 '2003.10.22.12.34.13',
 '2003.10.22.12.39.13',
 '2003.10.22.12.44.13',
 '2003.10.22.12.49.13']